# **Exploratory Data Analysis (EDA): Исследовательский анализ данных**

## **Введение в EDA**
EDA — это искусство задавать правильные вопросы данным. Это первый и самый важный этап работы с данными, где мы:
- 🕵️‍♂️ **Знакомимся** с данными
- 🔍 **Выявляем** закономерности и аномалии
- 📊 **Визуализируем** ключевые характеристики
- 💡 **Формулируем** гипотезы для дальнейшего анализа

**Цели EDA:**
1. Понимание структуры данных
2. Обнаружение выбросов и ошибок
3. Выявление взаимосвязей между переменными
4. Проверка предположений для моделей

## **🟢 Базовый уровень: Статистический анализ данных**

### **1.1 Первичное знакомство с данными**

In [ ]:
import pandas as pd
import seaborn as sns

# Загрузка данных
df = sns.load_dataset('titanic')

# Основная информация
print("Размер данных:", df.shape)
print("\nПервые 5 строк:")
display(df.head())

print("\nТипы данных:")
display(df.dtypes)

print("\nПропуски:")
display(df.isnull().sum())

### **1.2 Описательная статистика**

In [ ]:
# Для числовых переменных
print("Описательная статистика числовых колонок:")
display(df.describe())

# Для категориальных переменных
print("\nОписательная статистика категориальных колонок:")
display(df.describe(include=['object']))

### **1.3 Простые визуализации**

In [ ]:
import matplotlib.pyplot as plt

# Гистограмма распределения
plt.figure(figsize=(10, 6))
sns.histplot(df['age'].dropna(), kde=True, bins=20)
plt.title('Распределение возраста пассажиров')
plt.xlabel('Возраст')
plt.ylabel('Частота')
plt.show()

# Boxplot для обнаружения выбросов
plt.figure(figsize=(10, 6))
sns.boxplot(x=df['fare'])
plt.title('Распределение стоимости билетов')
plt.xlabel('Стоимость билета')
plt.ylabel('Частота')
plt.show()

### **1.4 Анализ категориальных переменных**

In [ ]:
# Мозаичный график
from statsmodels.graphics.mosaicplot import mosaic
mosaic(df, ['gender', 'target'], title='Распределение таргета по полу') if 'target' in df.columns else print('Колонка target отсутствует')
plt.show()

# Тепловая карта частот
cross_tab = pd.crosstab(df['city'], df['response']) if all(col in df.columns for col in ['city', 'response']) else None
if cross_tab is not None:
    sns.heatmap(cross_tab, annot=True, fmt='d', cmap='Blues')
    plt.title('Тепловая карта частот')
    plt.show()

## **🟡 Продвинутый уровень: Анализ взаимосвязей**

### **2.1 Корреляционный анализ**

In [ ]:
# Матрица корреляций
plt.figure(figsize=(12, 8))
corr_matrix = df.corr(numeric_only=True)
sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', fmt=".2f")
plt.title('Матрица корреляций')
plt.show()

# Парные взаимосвязи
sns.pairplot(df[['age', 'fare', 'pclass']].dropna())
plt.suptitle('Парные распределения', y=1.02)
plt.show()

### **2.2 Автоматизированный EDA**

In [ ]:
# Установка библиотек (раскомментировать при первом запуске)
# !pip install pandas-profiling sweetviz

# Pandas Profiling
from pandas_profiling import ProfileReport
profile = ProfileReport(df, title='Автоматический EDA отчет Titanic')
profile.to_file('titanic_report.html')

# SweetViz
import sweetviz as sv
report = sv.analyze(df)
report.show_html('sweetviz_report.html')

### **2.3 Временные паттерны**

In [ ]:
# Анализ трендов
if 'date' in df.columns and 'sales' in df.columns:
    df.resample('W')['sales'].mean().plot(
        kind='area', 
        alpha=0.3, 
        title='Продажи по неделям'
    )
    plt.show()

# Сезонная декомпозиция
from statsmodels.tsa.seasonal import seasonal_decompose
result = seasonal_decompose(df['fare'].dropna(), model='additive', period=365) if 'fare' in df.columns else None
if result:
    result.plot()
    plt.show()

## **🔴 Экспертный уровень: Продвинутые техники EDA**

### **3.1 Анализ мультиколлинеарности**

In [ ]:
from statsmodels.stats.outliers_influence import variance_inflation_factor
import numpy as np

# Расчет VIF
X = df.select_dtypes(include=[np.number]).dropna()
vif_data = pd.DataFrame()
vif_data["feature"] = X.columns
vif_data["VIF"] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
print(vif_data.sort_values('VIF', ascending=False))

# Визуализация зависимостей
sns.pairplot(X[['age', 'fare', 'pclass']], kind='reg', plot_kws={'line_kws':{'color':'red'}})
plt.show()

### **3.2 Нелинейные зависимости**

In [ ]:
# Mutual Information
from sklearn.feature_selection import mutual_info_classif
y = df['survived'] if 'survived' in df.columns else None
if y is not None and not X.empty:
    mi = mutual_info_classif(X, y)
    mi_series = pd.Series(mi, index=X.columns)
    mi_series.sort_values(ascending=False).plot(kind='barh')
    plt.title('Mutual Information')
    plt.show()

# Частичные зависимости
from sklearn.inspection import PartialDependenceDisplay
if 'model' in locals():
    PartialDependenceDisplay.from_estimator(model, X, features=[0, 1], grid_resolution=20)
    plt.show()

### **3.3 Большие данные**

In [ ]:
# Dask для анализа
from dask.dataframe import from_pandas
ddf = from_pandas(df, npartitions=4)
ddf_grouped = ddf.groupby('category')['value'].mean().compute() if 'category' in df.columns else None
print(ddf_grouped)

# Оптимизированная корреляция
corr = ddf.corr().compute()
print(corr)

## **📊 Чеклист по уровням**
| Уровень | Навыки |
|---------|--------|
| 🟢 | Гистограммы, boxplot, корреляционные матрицы |
| 🟡 | 3D-визуализация, автоматизированные отчеты, временные паттерны |
| 🔴 | VIF, частичные зависимости, анализ больших данных |

## **⚠️ Антипаттерны**
1. Игнорирование нормализации при сравнении переменных
2. Неправильная интерпретация корреляции как причинно-следственной связи
3. Анализ без domain knowledge (например, неучет бизнес-логики)
4. Перегрузка визуализаций (более 5-7 переменных на графике)

## **🚀 Продвинутые советы**

In [ ]:
# Авто-VIZ для больших датасетов
import dtale
dtale.show(df).open_browser()

# Кластеризация для EDA
from sklearn.cluster import KMeans
df['cluster'] = KMeans(n_clusters=5).fit_predict(X)
sns.scatterplot(x='x', y='y', hue='cluster', data=df)
plt.title('Кластеры')
plt.show()

# Проверка гипотез
from scipy.stats import ttest_ind
group1 = df[df['group'] == 'A']['value'] if 'group' in df.columns else None
group2 = df[df['group'] == 'B']['value'] if group1 is not None else None
if group1 is not None and group2 is not None:
    print(ttest_ind(group1, group2))

## **📈 Практический пример: Анализ Titanic**

In [ ]:
# 🟢 Базовый EDA
print(df[['Age', 'Fare']].describe())
sns.heatmap(df[['Age', 'Fare', 'Survived']].corr(), annot=True)
plt.title('Базовая корреляция')
plt.show()

# 🔴 Экспертный анализ
from sklearn.decomposition import PCA
pca = PCA(n_components=2)
df[['pca1', 'pca2']] = pca.fit_transform(X)
sns.scatterplot(x='pca1', y='pca2', hue='Survived', data=df)
plt.title('PCA-анализ')
plt.show()